In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.stats import chi2_contingency, f_oneway
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [175]:
print(sys.version)

3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:52:10) 
[Clang 14.0.6 ]


In [176]:

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import mutual_info_classif


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [177]:
df_train = pd.read_csv("GUIDE_Train.csv", nrows=80000)
df_test = pd.read_csv("GUIDE_Test.csv", nrows=20000)

In [178]:
df_train.head(5)

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [179]:
df_train = df_train.drop(columns = ["Id","OrgId","IncidentId","AlertId","DetectorId","DeviceId"])
df_test = df_test.drop(columns = ["Id","OrgId","IncidentId","AlertId","DetectorId","DeviceId","Usage"])

In [180]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           80000 non-null  object 
 1   AlertTitle          80000 non-null  int64  
 2   Category            80000 non-null  object 
 3   MitreTechniques     33954 non-null  object 
 4   IncidentGrade       79574 non-null  object 
 5   ActionGrouped       478 non-null    object 
 6   ActionGranular      478 non-null    object 
 7   EntityType          80000 non-null  object 
 8   EvidenceRole        80000 non-null  object 
 9   Sha256              80000 non-null  int64  
 10  IpAddress           80000 non-null  int64  
 11  Url                 80000 non-null  int64  
 12  AccountSid          80000 non-null  int64  
 13  AccountUpn          80000 non-null  int64  
 14  AccountObjectId     80000 non-null  int64  
 15  AccountName         80000 non-null  int64  
 16  Devi

In [181]:

#missing_values = df_train.isnull().sum()
#missing_percentages = 100 * missing_values / len(df_train)

# Display columns with missing values
#missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
#print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))


In [182]:
def drop_column_with_over_50_percent_missing(df):
    missing_values = df.isnull().sum()
    missing_percentages = 100 * missing_values / len(df)

    # Display columns with missing values
    missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))
    
    
    list = missing_data[missing_data.Percent > 50].index.tolist()
    df_drop = df.drop(columns = list)
    return df_drop


df_train_drop = drop_column_with_over_50_percent_missing(df_train)
df_test_drop = drop_column_with_over_50_percent_missing(df_test)

                   Total   Percent
ResourceType       79925  99.90625
ActionGrouped      79522  99.40250
ActionGranular     79522  99.40250
ThreatFamily       79361  99.20125
EmailClusterId     79192  98.99000
AntispamDirection  78573  98.21625
Roles              78178  97.72250
SuspicionLevel     67912  84.89000
LastVerdict        61323  76.65375
MitreTechniques    46046  57.55750
IncidentGrade        426   0.53250
                   Total  Percent
ActionGrouped      19988   99.940
ActionGranular     19988   99.940
ResourceType       19984   99.920
ThreatFamily       19862   99.310
EmailClusterId     19808   99.040
AntispamDirection  19601   98.005
Roles              19449   97.245
SuspicionLevel     16897   84.485
LastVerdict        15228   76.140
MitreTechniques    11081   55.405


In [183]:
df_train_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Timestamp           80000 non-null  object
 1   AlertTitle          80000 non-null  int64 
 2   Category            80000 non-null  object
 3   IncidentGrade       79574 non-null  object
 4   EntityType          80000 non-null  object
 5   EvidenceRole        80000 non-null  object
 6   Sha256              80000 non-null  int64 
 7   IpAddress           80000 non-null  int64 
 8   Url                 80000 non-null  int64 
 9   AccountSid          80000 non-null  int64 
 10  AccountUpn          80000 non-null  int64 
 11  AccountObjectId     80000 non-null  int64 
 12  AccountName         80000 non-null  int64 
 13  DeviceName          80000 non-null  int64 
 14  NetworkMessageId    80000 non-null  int64 
 15  RegistryKey         80000 non-null  int64 
 16  RegistryValueName   80

In [184]:
def fill_missing_value(df, column):
    df[column].fillna(df[column].mode()[0], inplace = True)

fill_missing_value(df_train_drop, 'IncidentGrade')
fill_missing_value(df_test_drop, 'IncidentGrade')

/var/folders/dt/7vnpb4hn01d19x7dfz27zs5h0000gn/T/ipykernel_88547/2763356033.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace = True)


In [185]:
numerical_columns = df_train_drop.select_dtypes(include=['int64']).columns

In [186]:
cat_columns = df_train_drop.select_dtypes(include=['object']).columns

In [187]:
df_train_drop[numerical_columns].nunique().sort_values(ascending=False)

AccountUpn            23228
AccountName           15778
AccountSid            15055
AccountObjectId       14926
IpAddress             11286
AlertTitle            10147
NetworkMessageId       9114
FileName               4922
DeviceName             4259
Url                    3613
Sha256                 3264
FolderPath             2841
City                    745
State                   342
CountryCode             100
ApplicationName          98
ApplicationId            70
ResourceIdName           64
RegistryKey              52
RegistryValueData        36
RegistryValueName        27
OSVersion                13
OAuthApplicationId       12
OSFamily                  4
dtype: int64

In [188]:
df_train_drop[cat_columns].nunique().sort_values(ascending=False)

Timestamp        65317
EntityType          24
Category            18
IncidentGrade        3
EvidenceRole         2
dtype: int64

In [189]:
df_train_drop[numerical_columns]

,AlertTitle,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,...,ApplicationName,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City
0,6,138268,27,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,31,6,3
1,43,138268,360606,160396,22406,23032,22795,24887,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
2,298,138268,360606,68652,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
3,2,138268,360606,13,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
4,74,138268,360606,160396,449,592,440,479,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,124,138268,360606,160396,13963,23237,13825,15203,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
79996,0,138268,14,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,14,32,30
79997,2,0,360606,160396,441377,673934,425863,453297,153085,529644,...,3421,881,0,4,3586,5,66,242,1445,10630
79998,19,138268,360606,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630


In [190]:
df_train_drop.columns.unique()

Index(['Timestamp', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType',
       'EvidenceRole', 'Sha256', 'IpAddress', 'Url', 'AccountSid',
       'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName',
       'NetworkMessageId', 'RegistryKey', 'RegistryValueName',
       'RegistryValueData', 'ApplicationId', 'ApplicationName',
       'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName',
       'OSFamily', 'OSVersion', 'CountryCode', 'State', 'City'],
      dtype='object')

In [191]:
def timestamp_transform(df):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['DayOfWeek'] = df['Timestamp'].dt.day_name() 
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

timestamp_transform(df_train_drop)
timestamp_transform(df_test_drop)

In [192]:
import pandas as pd
from scipy.stats import chi2_contingency, f_oneway
from sklearn.feature_selection import mutual_info_classif

# Significance threshold
alpha = 0.05

# Chi-Square Test for Categorical Features
def chi_square_test(feature, target):
    contingency_table = pd.crosstab(df_train_drop[feature], df_train_drop[target])
    chi2, p, dof, ex = chi2_contingency(contingency_table)
    return p

# ANOVA Test for Numerical Features
def anova_test(feature, target):
    groups = df_train_drop.groupby(target)[feature].apply(list)
    f_stat, p_value = f_oneway(*groups)
    return p_value

# Mutual Information for Numerical Features
def mutual_information(feature, target):
    mi = mutual_info_classif(df_train_drop[[feature]], df_train_drop[target])
    return mi[0]

chi_square_results = {}
anova_results = {}
mutual_info_results = {}

insignificant_features = []

for feature in df_train_drop.columns:
    if feature != 'IncidentGrade':
        if isinstance(df_train_drop[feature].dtype, pd.CategoricalDtype) or pd.api.types.is_object_dtype(df_train_drop[feature]):
            p_value = chi_square_test(feature, 'IncidentGrade')
            chi_square_results[feature] = p_value
            if p_value > alpha:
                insignificant_features.append((feature, 'Chi-Square', p_value))
        elif pd.api.types.is_numeric_dtype(df_train_drop[feature]):
            p_value = anova_test(feature, 'IncidentGrade')
            mi_score = mutual_information(feature, 'IncidentGrade')
            anova_results[feature] = p_value
            mutual_info_results[feature] = mi_score
            if p_value > alpha:
                insignificant_features.append((feature, 'ANOVA', p_value))

# Sort results
sorted_chi_square_results = sorted(chi_square_results.items(), key=lambda item: item[1], reverse=True)
sorted_anova_results = sorted(anova_results.items(), key=lambda item: item[1], reverse=True)
sorted_mi_results = sorted(mutual_info_results.items(), key=lambda item: item[1], reverse=True)

# Print insignificant features
if insignificant_features:
    print("Insignificant Features:")
    for feature, test, p_value in insignificant_features:
        print(f"Feature: {feature}, Test: {test}, P-Value: {p_value}, mutual info: {mi_score}")
else:
    print("All features are significant at the alpha level of", alpha)

# Optionally, print or return the sorted significant results
# print(sorted_chi_square_results, sorted_anova_results, sorted_mi_results)


All features are significant at the alpha level of 0.05


In [193]:
df_train_significant = df_train_drop.drop(columns = ["RegistryValueName","RegistryValueData"])
df_test_significant = df_test_drop.drop(columns = ["RegistryValueName","RegistryValueData"])

In [194]:
df_train_significant.describe()

,AlertTitle,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,...,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City,IsWeekend
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,...,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,2988.806163,128842.100925,285704.332188,150206.525187,351865.257312,463937.604137,340368.030275,356328.680713,143382.313025,480241.190275,...,880.829550,262685.895275,107791.842325,3582.587825,4.900762,64.686513,223.453150,1350.289712,9928.904375,0.170475
std,11570.313793,33763.267413,141306.787762,37734.673342,166961.821928,290493.688878,160428.266161,174922.737324,35773.423624,141580.775885,...,11.867501,80965.362319,31952.372872,106.229823,0.696237,9.210351,63.147289,353.165621,2619.128838,0.376052
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,138268.000000,360606.000000,160396.000000,441377.000000,91734.500000,425863.000000,449033.500000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
50%,11.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
75%,186.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
max,113151.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,1.000000


Encoding

In [195]:
df_train_significant.nunique().sort_values(ascending=False)

Timestamp             65317
AccountUpn            23228
AccountName           15778
AccountSid            15055
AccountObjectId       14926
IpAddress             11286
AlertTitle            10147
NetworkMessageId       9114
FileName               4922
DeviceName             4259
Url                    3613
Sha256                 3264
FolderPath             2841
City                    745
State                   342
CountryCode             100
ApplicationName          98
ApplicationId            70
ResourceIdName           64
RegistryKey              52
EntityType               24
Category                 18
OSVersion                13
OAuthApplicationId       12
DayOfWeek                 7
OSFamily                  4
IncidentGrade             3
EvidenceRole              2
IsWeekend                 2
dtype: int64

In [196]:
df_train_X = df_train_significant.drop(columns=["IncidentGrade"])
df_train_Y = df_train_significant["IncidentGrade"]
df_test_X = df_test_significant.drop(columns=["IncidentGrade"])
df_test_Y = df_test_significant["IncidentGrade"]

In [197]:
def target_encode(df, n_bins):
    """
    Preprocesses the data by performing target encoding, frequency encoding, 
    and hashing for high cardinality columns.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame to preprocess.
    - low_cardinality_columns (list): List of columns with low cardinality for frequency encoding.
    - high_cardinality_columns (list): List of columns with high cardinality for hashing.
    - target_column (str): The name of the target variable for target encoding.
    - n_bins (int, optional): Number of bins for hashing high cardinality columns. Default is 100.
    
    Returns:
    - pd.DataFrame: The preprocessed DataFrame.
    """
    
    low_cardinality_columns = df.drop(columns = ["Timestamp"]).nunique()[df.nunique() <= 10].index
    high_cardinality_columns = df.drop(columns = ["Timestamp"]).nunique()[df.nunique() > 10].index

    # Copy the DataFrame to avoid altering the original data
    df_encoded = df.copy()
    
    # Drop the 'Timestamp' column
    if 'Timestamp' in df_encoded.columns:
        df_encoded = df_encoded.drop('Timestamp', axis=1)
    
    # Target encoding
    encoder = TargetEncoder()
    
    # Perform frequency encoding for low cardinality columns
    for column in low_cardinality_columns:
        df_encoded[column] = df_encoded[column].map(df_encoded[column].value_counts())
    
    # Perform hashing for high cardinality columns
    for column in high_cardinality_columns:
        df_encoded[column] = df_encoded[column].apply(lambda x: np.abs(hash(str(x))) % n_bins)
    
    return df_encoded

In [198]:
X_train_encoded = target_encode(df_train_X, 100)
X_test_encoded = target_encode(df_test_X, 100)

In [199]:
X_test_encoded.nunique().sort_values(ascending=False)

AlertTitle            100
AccountSid            100
FileName              100
NetworkMessageId      100
DeviceName            100
AccountName           100
AccountUpn            100
AccountObjectId       100
Url                   100
IpAddress             100
Sha256                100
FolderPath            100
City                   95
State                  83
CountryCode            51
ApplicationName        27
EntityType             19
ApplicationId          18
ResourceIdName         16
RegistryKey            14
Category               14
OSVersion              12
DayOfWeek               7
OSFamily                4
OAuthApplicationId      3
EvidenceRole            2
IsWeekend               2
dtype: int64

In [200]:
# Scaling

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled



X_train_scaled, X_test_scaled = scale_data(X_train_encoded, X_test_encoded)

# Model Selection

In [206]:
def model_comparison(X_train_scaled, X_test_scaled, df_train_Y, df_test_Y):
    results = {}

    # Random Forest
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train_scaled, df_train_Y)
    y_pred_rf = rf_model.predict(X_test_scaled)

    rf_report = classification_report(df_test_Y, y_pred_rf, output_dict=True)
    results['RandomForest'] = rf_report['macro avg']
    results['RandomForest'].update(rf_report['weighted avg'])
    results['RandomForest']['accuracy'] = rf_report['accuracy']

    #print("Random Forest Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_rf))

    # Gradient Boost
    gb_model = GradientBoostingClassifier(random_state=42)
    gb_model.fit(X_train_scaled, df_train_Y)
    y_pred_gb = gb_model.predict(X_test_scaled)

    gb_report = classification_report(df_test_Y, y_pred_gb, output_dict=True)
    results['GradientBoost'] = gb_report['macro avg']
    results['GradientBoost'].update(gb_report['weighted avg'])
    results['GradientBoost']['accuracy'] = gb_report['accuracy']

    #print("Gradient Boosting Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_gb))

    # XGBoost
    xgb_model = XGBClassifier(random_state=42)
    le = LabelEncoder()
    label_train_y = le.fit_transform(df_train_Y)
    label_test_y = le.transform(df_test_Y)

    xgb_model.fit(X_train_scaled, label_train_y)
    y_pred_xgb = xgb_model.predict(X_test_scaled)

    xgb_report = classification_report(label_test_y, y_pred_xgb, output_dict=True)
    results['XGBoost'] = xgb_report['macro avg']
    results['XGBoost'].update(xgb_report['weighted avg'])
    results['XGBoost']['accuracy'] = xgb_report['accuracy']

    #print("XGBoost Classifier Report:")
    #print(classification_report(label_test_y, y_pred_xgb))

    # LightGBM
    lgbm_model = LGBMClassifier(random_state=42)
    lgbm_model.fit(X_train_scaled, df_train_Y)
    y_pred_lgbm = lgbm_model.predict(X_test_scaled)

    lgbm_report = classification_report(df_test_Y, y_pred_lgbm, output_dict=True)
    results['LightGBM'] = lgbm_report['macro avg']
    results['LightGBM'].update(lgbm_report['weighted avg'])
    results['LightGBM']['accuracy'] = lgbm_report['accuracy']

    #print("LightGBM Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_lgbm))

    # CatBoost
    catboost_model = CatBoostClassifier(random_state=42, verbose=0)
    catboost_model.fit(X_train_scaled, df_train_Y)
    y_pred_catboost = catboost_model.predict(X_test_scaled)

    catboost_report = classification_report(df_test_Y, y_pred_catboost, output_dict=True)
    results['CatBoost'] = catboost_report['macro avg']
    results['CatBoost'].update(catboost_report['weighted avg'])
    results['CatBoost']['accuracy'] = catboost_report['accuracy']

    #print("CatBoost Classifier Report:")
    #print(classification_report(df_test_Y, y_pred_catboost))

    # Convert the results dictionary into a DataFrame
    results_df = pd.DataFrame(results)

    # Transpose the DataFrame for better readability
    results_df = results_df.T

    # Display the combined DataFrame
    return results_df

In [207]:
model_comparison(X_train_scaled, X_test_scaled, df_train_Y, df_test_Y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006242 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1127
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 26
[LightGBM] [Info] Start training from score -0.831031
[LightGBM] [Info] Start training from score -1.532188
[LightGBM] [Info] Start training from score -1.054583


,precision,recall,f1-score,support,accuracy
RandomForest,0.686257,0.67100,0.649266,20000.0,0.67100
GradientBoost,0.699903,0.66380,0.624061,20000.0,0.66380
XGBoost,0.703958,0.67885,0.669383,20000.0,0.67885
LightGBM,0.724249,0.69375,0.669625,20000.0,0.69375
CatBoost,0.701282,0.68900,0.669541,20000.0,0.68900


In [208]:



# Assume X_train_scaled and df_train_Y are already defined
# X_train_scaled: Scaled features for training
# df_train_Y: Target variable for training

# Initialize K-Fold cross-validation with 5 folds


def cross_validation(X_train_scaled, df_train_Y, kf):
    # Initialize a dictionary to hold the cross-validation results
    cv_results = {}

    # Random Forest
    rf_model = RandomForestClassifier(random_state=42)
    rf_scores = cross_val_score(rf_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['RandomForest'] = rf_scores
    #print(f"Random Forest CV Accuracy: {rf_scores.mean():.4f} (+/- {rf_scores.std():.4f})")

    # Gradient Boosting
    gb_model = GradientBoostingClassifier(random_state=42)
    gb_scores = cross_val_score(gb_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['GradientBoost'] = gb_scores
    #print(f"Gradient Boosting CV Accuracy: {gb_scores.mean():.4f} (+/- {gb_scores.std():.4f})")

    # XGBoost
    xgb_model = XGBClassifier(random_state=42)
    le = LabelEncoder()
    label_train_y = le.fit_transform(df_train_Y)

    xgb_scores = cross_val_score(xgb_model, X_train_scaled, label_train_y, cv=kf, scoring='accuracy')
    cv_results['XGBoost'] = xgb_scores
    #print(f"XGBoost CV Accuracy: {xgb_scores.mean():.4f} (+/- {xgb_scores.std():.4f})")

    # LightGBM
    lgbm_model = LGBMClassifier(random_state=42)
    lgbm_scores = cross_val_score(lgbm_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['LightGBM'] = lgbm_scores
    #print(f"LightGBM CV Accuracy: {lgbm_scores.mean():.4f} (+/- {lgbm_scores.std():.4f})")

    # CatBoost
    catboost_model = CatBoostClassifier(random_state=42, verbose=0)
    catboost_scores = cross_val_score(catboost_model, X_train_scaled, df_train_Y, cv=kf, scoring='accuracy')
    cv_results['CatBoost'] = catboost_scores
    #print(f"CatBoost CV Accuracy: {catboost_scores.mean():.4f} (+/- {catboost_scores.std():.4f})")

    # Convert the cross-validation results into a DataFrame for easier comparison
    cv_results_df = pd.DataFrame(cv_results)

    # Transpose the DataFrame for better readability
    cv_results_df = cv_results_df.T

    # Add a column for the mean accuracy and standard deviation
    cv_results_df['Mean Accuracy'] = cv_results_df.mean(axis=1)
    cv_results_df['Std Dev'] = cv_results_df.std(axis=1)

    # Display the combined cross-validation results
    #print(cv_results_df)
    return cv_results_df

In [209]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = cross_validation(X_train_scaled, df_train_Y, kf)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006241 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1111
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 26
[LightGBM] [Info] Start training from score -0.830256
[LightGBM] [Info] Start training from score -1.534577
[LightGBM] [Info] Start training from score -1.054072
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004730 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1122
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 26
[LightGBM] [Info] Start training from score -0.829146
[LightGBM] [Info] Start training from score -1.530020
[LightGBM] [Info] Start 

In [210]:
cv_results

,0,1,2,3,4,Mean Accuracy,Std Dev
RandomForest,0.699313,0.698187,0.704750,0.701688,0.703750,0.701537,0.002506
GradientBoost,0.658563,0.651375,0.656625,0.658312,0.654062,0.655788,0.002728
XGBoost,0.701500,0.700125,0.700000,0.701313,0.698812,0.700350,0.000978
LightGBM,0.696313,0.695375,0.696375,0.698688,0.694688,0.696288,0.001354
CatBoost,0.699063,0.696313,0.698750,0.699313,0.697187,0.698125,0.001170
